In [1]:
expid='MC_O_20250411_000008'
sensor='R00_SG1'

In [2]:
from lsst.resources import ResourcePath
from astropy.io import fits
import os
import numpy
from astropy.table import Table
from dateutil.parser import parse
import re
import json
import matplotlib.pyplot as plt

#MC_C_20240918_000019_R40_SG1_guider.fits
p = re.compile( rf'MC_([OC])_(\d*)_(\d*)_(R.._S..)_guider.fits' )


In [3]:
def examineone(fname):
    oc, date, seqnum, sensor = p.match(fname).groups()
    path = f's3://embargo@rubin-summit/LSSTCam/{date}/MC_{oc}_{date}_{seqnum}/{fname}'
    rp=ResourcePath(path)
    with rp.open(mode="rb") as f:
        hdu_list = fits.open(f)
    hdus = hdu_list[2::2]  # for the mode with raw stamps
#    hdus = hdu_list[1:]   # for the mode without raw stamps
    timing = [ parse(hdu.header['STMPTIME']) for hdu in hdus ]
    mean = [ numpy.mean(hdu.data) for hdu in hdus ]
    stdnoise = numpy.mean(list(map( lambda x: numpy.std(x.data[1:-1,1:-1]), hdus))[1:]) 
    stdstdnoise = numpy.std(list(map( lambda x: numpy.std(x.data[1:-1,1:-1]), hdus))[1:]) 

    hz = 1/numpy.array(list(map(lambda x: x.total_seconds(), numpy.diff(numpy.array(list(map( lambda x: parse(x.header['STMPTIME']), hdus) )[1:]))))).mean()
    
    return {
        's3path': rp,
        'basename': rp.abspath().basename(),
        'noise': stdnoise,
        'std(noise)': stdstdnoise,        
        'Hz': hz,
        'mean': mean,
        'timing': timing
    }
    

In [ ]:
table=Table(
    list(map( lambda x: examineone(x), [
        f'{expid}_R00_SG0_guider.fits',
        f'{expid}_R00_SG1_guider.fits',
        f'{expid}_R04_SG0_guider.fits',
        f'{expid}_R04_SG1_guider.fits',
        f'{expid}_R40_SG0_guider.fits',
        f'{expid}_R40_SG1_guider.fits',
        f'{expid}_R44_SG0_guider.fits',
        f'{expid}_R44_SG1_guider.fits'
    ])
       )
)
table['basename','noise','std(noise)','Hz','mean','timing']

In [ ]:
for row in table:
    plt.plot(row['timing'],row['mean'],'.',label=row['basename'])
plt.legend()

In [ ]:
rp = ResourcePath(table[1]['s3path'].geturl().replace('.fits','.json'))
with rp.open(mode="r") as f:
    j=json.load(f)
print(json.dumps(j,indent=2))

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from astropy.io import fits
from IPython.display import HTML
%matplotlib widget

s3file = table[table['basename']==f'{expid}_{sensor}_guider.fits']['s3path'][0]
with s3file.open(mode='rb') as f:
    fh = fits.open(f)

fig, ax = plt.subplots()

frames = [[plt.imshow(img.data, animated=True)] for img in fh[2::2]]
plt.colorbar()
plt.title(s3file)
ani = animation.ArtistAnimation(fig, frames, interval=120, blit=True, repeat_delay=1000)

HTML(ani.to_html5_video())
#plt.show()